https://www.youtube.com/watch?v=jrR_WfgmWEw

In [1]:
%%writefile nginx.conf
events {
    worker_connections 1024;
}

http {
    server {
        listen 80;
        server_name javiergimenez.es www.javiergimenez.es;
        
        location /.well-known/acme-challenge/ {
            root /letsencrypt/;
        }

        location / {
            root /usr/share/nginx/html;
            index index.html index.htm;
        }
    }



    # server {
    #   listen 443 ssl default_server;
    #   listen [::]:443 ssl default_server ;
    #   server_name javiergimenez.es;
    #   ssl_certificate /etc/letsencrypt/live/javiergimenez.es/fullchain.pem;
    #   ssl_certificate_key /etc/letsencrypt/live/javiergimenez.es/privkey.pem;
    #   root /usr/share/nginx/html/;

    #   location / {
    #     gzip off;
    #     root /usr/share/nginx/html/;
    #     index  index.html;
    #   }

    # }


}

Writing nginx.conf


In [2]:
%%bash
mkdir certs
mkdir html

```bash
mkdir certs
mkdir html

In [3]:
%%writefile ./html/index.html
<html>
<head>
    <title>Test</title>
</head>
<body>
    <h1>Test</h1>
    <p>Test</p>
</html>

Writing ./html/index.html


```bash
docker run -d --name nginx -v ${PWD}/nginx.conf:/etc/nginx/nginx.conf -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt -v ${PWD}/html:/usr/share/nginx/html -p 80:80 -p 443:443 nginx

In [4]:
!curl ifconfig.co

83.138.44.54


```bash
curl ifconfig.co

<h2> Una vez que funciona el servidor web (sin https) se procede a usar certbot para obtener un certificado SSL.</H2>


```bash
## este docker su punto de ejecución es el comando "cerbot" hay que añadir subcomando y opciones si necesarias
# docker run -it --rm --name certbot -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt certbot/certbot [subcomando][opciones]

docker run -it --rm --name certbot -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt certbot/certbot certonly --webroot

<h2>Ahora antes de las pruebas automáticas de verificación, cerbot te pide el dominio "javiergimenez.es", el correo (si se quiere), y el directorio donde se realizaran las pruebas automáticas "/letsencrypt"</h2>

Status: Downloaded newer image for certbot/certbot:latest
Saving debug log to /var/log/letsencrypt/letsencrypt.log
Enter email address (used for urgent renewal and security notices)
 (Enter 'c' to cancel): javiergimenezmoya@gmail.com

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Please read the Terms of Service at
https://letsencrypt.org/documents/LE-SA-v1.4-April-3-2024.pdf. You must agree in
order to register with the ACME server. Do you agree?
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
(Y)es/(N)o: Y

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Would you be willing, once your first certificate is successfully issued, to
share your email address with the Electronic Frontier Foundation, a founding
partner of the Let's Encrypt project and the non-profit organization that
develops Certbot? We'd like to send you email about our work encrypting the web,
EFF news, campaigns, and ways to support digital freedom.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
(Y)es/(N)o: Y
Account registered.
Please enter the domain name(s) you would like on your certificate (comma and/or
space separated) (Enter 'c' to cancel): ia.javiergimenez.es
Requesting a certificate for ia.javiergimenez.es
Input the webroot for ia.javiergimenez.es: (Enter 'c' to cancel): /letsencrypt     

Successfully received certificate.
Certificate is saved at: /etc/letsencrypt/live/ia.javiergimenez.es/fullchain.pem
Key is saved at:         /etc/letsencrypt/live/ia.javiergimenez.es/privkey.pem
This certificate expires on 2024-08-04.
These files will be updated when the certificate renews.

NEXT STEPS:
- The certificate will need to be renewed before it expires. Certbot can automatically renew the certificate in the background, but you may need to take steps to enable that functionality. See https://certbot.org/renewal-setup for instructions.
We were unable to subscribe you the EFF mailing list because your e-mail address appears to be invalid. You can try again later by visiting https://act.eff.org.

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
If you like Certbot, please consider supporting our work by:
 * Donating to ISRG / Let's Encrypt:   https://letsencrypt.org/donate
 * Donating to EFF:                    https://eff.org/donate-le
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

<h2>Una vez que ya tenemos el certificado SSL ya podemos incorporar la configuración de https en el archivo de configuración de nginx.</H2>

In [5]:
%%writefile nginx.conf
events {
    worker_connections 1024;
}

http {
    server {
        listen 80;
        server_name javiergimenez.es www.ia.javiergimenez.es;
        
        location /.well-known/acme-challenge/ {
            root /letsencrypt/;
        }

        location / {
            root /usr/share/nginx/html;
            index index.html index.htm;
        }
    }



    server {
      listen 443 ssl default_server;
      listen [::]:443 ssl default_server ;
      server_name javiergimenez.es;
      ssl_certificate /etc/letsencrypt/live/javiergimenez.es/fullchain.pem;
      ssl_certificate_key /etc/letsencrypt/live/javiergimenez.es/privkey.pem;
      root /usr/share/nginx/html/;

      location / {
        gzip off;
        root /usr/share/nginx/html/;
        index  index.html;
      }

    }


}

Overwriting nginx.conf


```bash
docker container restart nginx

# renovaciones

Simulacro de renovación

docker run -it --rm --name certbot -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt certbot/certbot renew --dry-run

Renovación

```BASH
docker run -it --rm --name certbot -v ${PWD}:/letsencrypt -v ${PWD}/certs:/etc/letsencrypt certbot/certbot renew 

Los directorios deben de tener permisos al menos de lectura y ejecución

```bash
chmod 777 ./certs/live
chmod 777 ./certs/accounts
chmod 777 ./certs/archive

<h1>
No olvidar los portforwarding en el router para puerto 80 y 443

Tampoco olvidar abrirlos en el cortafuegos
</h1>

# Creación de un servidor flask con certificado SSL
https://chat.openai.com/share/b4add14f-7e2c-40cf-a15c-7e7c27bd1393

In [ ]:
%%writefile nginx.conf

events {
    worker_connections 1024;
}

http {
    server {
        listen 80;
        server_name javiergimenez.es www.javiergimenez.es;

        location /.well-known/acme-challenge/ {
            root /letsencrypt/;
        }

        location / {
            return 301 https://$server_name$request_uri;  # Redirigir todo el tráfico HTTP a HTTPS
        }
    }

    server {
        listen 443 ssl default_server;
        listen [::]:443 ssl default_server;
        server_name javiergimenez.es;
        ssl_certificate /etc/letsencrypt/live/javiergimenez.es/fullchain.pem;
        ssl_certificate_key /etc/letsencrypt/live/javiergimenez.es/privkey.pem;
        root /usr/share/nginx/html/;

        location / {
            gzip off;
            root /usr/share/nginx/html/;
            index index.html;
        }

        location /api/ {
            # proxy_pass http://ia.javiergimenez.es:5500;
            proxy_pass http://ia.javiergimenez:5500/api/;  # Redirigir las solicitudes /api a Flask
            proxy_http_version 1.1;
            proxy_set_header Upgrade $http_upgrade;
            proxy_set_header Connection "upgrade";
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header X-Forwarded-Proto $scheme;       
            add_header 'Access-Control-Allow-Origin' '*';  # Permite todas las solicitudes CORS
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS, PUT, DELETE';  # Métodos permitidos
   
        }
    }
}


Overwriting nginx.conf


In [ ]:
# %%writefile serverflask.py
# from flask import Flask
# app = Flask(__name__)

# @app.route('/api/hola')
# def hola():
#     return "Hola Mundo!"

# if __name__ == "__main__":
#     app.run(host='0.0.0.0', port=5500)


```bash
#crear el proceso de servidor web flask
python serverflask.py

```bash
#para ver PID del proceso por si lo queremos luego matar
pgrep -f "python serverflask.py"

```bash
#para ver el proceso en el contexto de red y puerto de escucha
ss -tulpn | grep 5500


In [8]:
%%writefile ./html/cliente.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hola Mundo App</title>
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
    <script>
    $(document).ready(function(){
        $("#btnHola").click(function(){
            $.ajax({
                url: '/api/hola',
                type: 'GET',
                success: function(response) {
                    alert(response);
                },
                error: function(error) {
                    console.log(error);
                }
            });
        });
    });
    </script>
</head>
<body>
    <h1>Hola Mundo App</h1>
    <button id="btnHola">Di Hola</button>
</body>
</html>


Writing ./html/cliente.html
